The basic feature selection methods are mostly about individual proprtis of features & how they interact with each other

RFE selects features based on how thy affect a particular model's performance

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2]:
df=pd.read_csv('ANSUR II MALE Public.csv',encoding='latin-1',nrows=1000)
df.head()

,subjectid,abdominalextensiondepthsitting,acromialheight,acromionradialelength,anklecircumference,axillaheight,balloffootcircumference,balloffootlength,biacromialbreadth,bicepscircumferenceflexed,...,Branch,PrimaryMOS,SubjectsBirthLocation,SubjectNumericRace,Ethnicity,DODRace,Age,Heightin,Weightlbs,WritingPreference
0,10027,266,1467,337,222,1347,253,202,401,369,...,Combat Arms,19D,North Dakota,1,NaN,1,41,71,180,Right hand
1,10032,233,1395,326,220,1293,245,193,394,338,...,Combat Support,68W,New York,1,NaN,1,35,68,160,Left hand
2,10033,287,1430,341,230,1327,256,196,427,408,...,Combat Support,68W,New York,2,NaN,2,42,68,205,Left hand
3,10092,234,1347,310,230,1239,262,199,401,359,...,Combat Service Support,88M,Wisconsin,1,NaN,1,31,66,175,Right hand
4,10093,250,1585,372,247,1478,267,224,435,356,...,Combat Service Support,92G,North Carolina,2,NaN,2,21,77,213,Right hand


In [3]:
df.shape

(1000, 108)

In [4]:
data=df.select_dtypes(include=np.number) #Comtaining only numeric columns
data.shape

(1000, 99)

In [5]:
X=data.drop(columns="Weightlbs",axis=1)
y=data["Weightlbs"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)
print(f"Train Data: {X_train.shape},{y_train.shape}")
print(f"Test Data: {X_test.shape},{y_test.shape}")

Train Data: (800, 98),(800,)
Test Data: (200, 98),(200,)


In [7]:
ss=StandardScaler()
scaled_train=ss.fit_transform(X_train)
scaled_test=ss.transform(X_test)

In [8]:
rf_model=RandomForestRegressor(random_state=24)
rf_model.fit(scaled_train,y_train)

RandomForestRegressor(random_state=24)

In [9]:
print(f"Score:{rf_model.score(scaled_test,y_test)} ")

Score:0.9525694773382329 


In [10]:
f_importance=pd.concat([pd.DataFrame(X_train.columns),pd.DataFrame(rf_model.feature_importances_)],axis=1)
f_importance.columns=["feature","importance"]
f_importance.sort_values(by='importance').reset_index(drop=True)

,feature,importance
0,DODRace,0.000036
1,Heightin,0.000120
2,crotchheight,0.000127
3,tibialheight,0.000130
4,acromionradialelength,0.000130
...,...,...
93,abdominalextensiondepthsitting,0.002206
94,bicepscircumferenceflexed,0.003395
95,buttockdepth,0.003969
96,bideltoidbreadth,0.014106


**Performing Recursive Feature Elimination**

In [11]:
rfe=RFE(estimator = RandomForestRegressor(random_state=24) , n_features_to_select= 10)
rfe.fit(scaled_train,y_train)

RFE(estimator=RandomForestRegressor(random_state=24), n_features_to_select=10)

In [12]:
rfe.support_

array([False, False, False, False, False, False, False, False, False,
        True, False,  True, False, False, False, False, False,  True,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False,  True, False, False, False, False, False, False])

In [15]:
sum(rfe.support_)

10

In [13]:
X_train_selected= rfe.transform(scaled_train)
print("X_train.shape:{}".format(scaled_train.shape))
print("X_train_selected.shape: {}".format(X_train_selected.shape))

X_train.shape:(800, 98)
X_train_selected.shape: (800, 10)


In [14]:
X_test_selected= rfe.transform(scaled_test)
X_test_selected.shape

(200, 10)

In [18]:
X_train_selected = rfe.transform(scaled_train)
print("X_train.shape:{}".format(scaled_train.shape))
print("X_train_selected.shape: {}".format(X_train_selected.shape))
X_test_selected= rfe.transform(scaled_test)
print("X_test.shape:{}".format(scaled_test.shape))
print("X_test_selected.shape: {}".format(X_test_selected.shape))

X_train.shape:(800, 98)
X_train_selected.shape: (800, 10)
X_test.shape:(200, 98)
X_test_selected.shape: (200, 10)


In [19]:
rf_model=RandomForestRegressor(random_state=24)
rf_model.fit(X_train_selected,y_train)

RandomForestRegressor(random_state=24)

In [20]:
print(f"Score:{rf_model.score(X_test_selected,y_test)} ")

Score:0.9531169389266686 


It is similiar to above if we use all the columns. This is hoe we can reduce the dimension by retaining the metric of the mode. Here we have used only 10 columns , so the computation is fast, less expensive & less chance of overfitting. It will reduce the curse of dimensionality